<a href="https://colab.research.google.com/github/beanbean0510/AWINLAB_homework/blob/main/70DogBreeds_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**深度學習（Deep Learning）**
- **資料集：**70 Dog Breeds-Image Data Set（https://www.kaggle.com/datasets/gpiosenka/70-dog-breedsimage-data-set）
- **指定類別：**"Airedale", "Beagle", "Bloodhound", "Bluetick", "Chihuahua", "Collie", "Dingo", " French Bulldog", " German Sheperd", " Malinois", " Newfoundland", " Pekinese", " Pomeranian", "Pug", "Vizsla" 共15項
- **使用環境：**Google CoLab


**首先連結 Google Drive 檔案**

In [4]:
### 將檔案連接至 Google Drive 雲端硬碟 ###
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
### 檢視雲端硬碟中的檔案路徑 確認檔案所在地 ###
from pathlib import Path

# 將所有在路徑 '/content/drive/MyDrive/AWINLAB/' 中的檔案印出，確認所需資料是否存在
for path in Path('/content/drive/MyDrive/AWINLAB/').glob("*"):
  print(path)

/content/drive/MyDrive/AWINLAB/dogs.csv
/content/drive/MyDrive/AWINLAB/test_data.xlsx
/content/drive/MyDrive/AWINLAB/valid
/content/drive/MyDrive/AWINLAB/train
/content/drive/MyDrive/AWINLAB/test


### **函式庫筆記**
1. ***numpy***：
  - 提供非常高效能的多維陣列(multi-dimensional array)數學函式庫
  - 可整合C/C++及Fortran的程式碼
  - 方便有用的線性代數(Linear Algebra)及傅立葉轉換(Fourier Transform)能力
  - 利用NumPy Array替代Python List
  - 可定義任意的數據型態(Data Type)，使得能輕易及無縫的與多種資料庫整合
2. ***matplotlib***
  - 為 python 的一個 2D 繪圖庫
  - 可用來繪圖、呈現圖表、表示數據
3. ***pandas***
  - 可以被用來執行強大的資料分析
  - 結合 numpy 的特性以及試算表和關連式資料庫的資料操作能力，可以用來對資料進行重構、切割、聚合及選擇子集合等操作
4. ***tensorflow***
  - 是一個免費的開源軟體程式庫，適用於機器學習和人工智慧
5. ***keras***
  - 是一個深度學習的開源函式庫
  - 能夠提供簡單而快速的原型設計，是個高度模組化的函式庫，可以讓用戶快速的去建造神經網路並訓練模型

In [6]:
### 導入所需的函式庫 ###
# 常用函式庫
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 機器學習、深度學習相關
import tensorflow as tf
from tensorflow import keras

# 導入 Sequential 用來搭建線性堆疊模型
from tensorflow.keras.models import Sequential

# 導入 keras 的層（layers）中所需的各種工具
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout , Input , BatchNormalization

# 導入 keras 的優化器（optimizers）中所需的各種工具
from tensorflow.keras.optimizers import RMSprop, Adam

In [20]:
### 引入所需資料 ###
df_train = '/content/drive/MyDrive/AWINLAB/train'
df_valid = '/content/drive/MyDrive/AWINLAB/valid'
df_test = '/content/drive/MyDrive/AWINLAB/test'

- ***ImageDataGenerator：***可利用現有的資料經過旋轉、翻轉、縮放…等方式增加更多的訓練資料

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 將圖像的像素縮放到指定的範圍內，一般會縮放至[0,1]之間
datagen = ImageDataGenerator(rescale = 1./255)

In [22]:
# 調整資料，將資料批次（batch）設為64、並將圖像像素（target）調整成224x224、分類標籤設為'categorical'利用one-hot轉換
ds_train = datagen.flow_from_directory(df_train, batch_size = 64, target_size = (224, 224),
                                       class_mode = 'categorical')

ds_valid = datagen.flow_from_directory(df_valid, batch_size = 64, target_size = (224, 224),
                                       class_mode = 'categorical')

ds_test = datagen.flow_from_directory(df_test, batch_size = 64, target_size = (224, 224),
                                       class_mode = 'categorical')

# 因為資料共有15個類別，故設為15
num_classes = 15

Found 1714 images belonging to 15 classes.
Found 150 images belonging to 15 classes.
Found 300 images belonging to 1 classes.


In [26]:
### 建立 CNN 模型 ###

# 指定圖像的尺寸和維度
input_shape = (224, 224, 3)

# 建立一個線性堆疊的模型，用於按順序堆疊神經網路層
model = Sequential()
# 增加輸入層，設定輸入格式為剛剛設定的 input_shape
model.add(Input(shape=input_shape))
# 加入第一層卷積層，過濾器大小設為3x3數量為32個，激勵函數設為'ReLU'
# ReLU 線性整流單元：會將所有負值轉換成零，避免卷積的過程將數字變成無限大或是趨近於0
將所有負值轉換成零，可以避免卷積的過程將數字變成無限大或是趨近於0
model.add(Conv2D(32, (3, 3), activation='relu'))
# 加入池化層：保留重要資訊，減少參數數量
model.add(MaxPooling2D((2, 2)))
# 加入批歸一化層：可將分散的數據統一，有助於減緩梯度消失，也可解決 Internal Covariate Shift 的問題及加速收斂
model.add(BatchNormalization())
# 重複添加上述結構，共三層
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
# 建立平坦層：將矩陣打平成一維的陣列作為輸入
model.add(Flatten())
# 建立隱藏層
model.add(Dense(128, activation='relu'))
# 防止 overfitting
model.add(Dropout(0.5))
# 建立輸出層：將輸入值轉化成各個類別的機率，機率最大的即是最有可能的類別
model.add(Dense(15, activation='softmax'))
# 編譯模型
model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

In [27]:
### 訓練模型 ###
# 訓練回數設為30回、每個批次樣本數為32
# verbose：設為0：執行過程中不輸出資訊；設為1：顯示訓練進度訊息；設為2：為每個 epoch 輸出一行紀錄（默認為1）
history = model.fit(ds_train, epochs = 30, validation_data = ds_valid , batch_size = 32 , verbose = 1)

Epoch 1/30
27/27 [==============================] - 473s 17s/step - loss: 6.5383 - accuracy: 0.1820 - val_loss: 3.2297 - val_accuracy: 0.0667
Epoch 2/30
27/27 [==============================] - 134s 5s/step - loss: 2.3733 - accuracy: 0.4726 - val_loss: 4.3080 - val_accuracy: 0.0667
Epoch 3/30
27/27 [==============================] - 135s 5s/step - loss: 1.6095 - accuracy: 0.5928 - val_loss: 6.5609 - val_accuracy: 0.0800
Epoch 4/30
27/27 [==============================] - 134s 5s/step - loss: 1.1285 - accuracy: 0.6902 - val_loss: 4.5176 - val_accuracy: 0.1133
Epoch 5/30
27/27 [==============================] - 133s 5s/step - loss: 0.9609 - accuracy: 0.7357 - val_loss: 7.0663 - val_accuracy: 0.0667
Epoch 6/30
27/27 [==============================] - 133s 5s/step - loss: 0.7104 - accuracy: 0.8098 - val_loss: 6.4222 - val_accuracy: 0.0667
Epoch 7/30
27/27 [==============================] - 133s 5s/step - loss: 0.5784 - accuracy: 0.8355 - val_loss: 10.6067 - val_accuracy: 0.0733
Epoch 8/30


In [28]:
### 利用 Testing set 評估模型 ###
loss_cnn, accuracy_cnn = model.evaluate(ds_test)
loss_cnn = loss_cnn*100
accuracy_cnn = accuracy_cnn*100
print('Test loss:', loss_cnn)
print('Test accuracy:', accuracy_cnn)

5/5 [==============================] - 47s 11s/step - loss: 229.1060 - accuracy: 0.0767
Test loss: 22910.604858398438
Test accuracy: 7.666666805744171
